# Plan de Despliegue: Sistema de Anotación de Video

> **Entrega 2 - Universidad ICESI**
>
> - **Modelo Final:** MLP con regularización
> - **Performance:** ~85% accuracy (post-corrección data leakage)
> - **Objetivo:** Presentar un plan completo para el despliegue del modelo en un entorno de producción.
---

## 🏆 Modelo a Desplegar

### Especificaciones Técnicas
- **Algoritmo:** Multi-Layer Perceptron (MLP) con capas de (100, 50) neuronas.
- **Input:** 19 características obtenidas de un PCA sobre los landmarks de MediaPipe.
- **Output:** Clasificación de 5 actividades.
- **Accuracy:** 85% (estimación realista, corregido el data leakage).
- **Tiempo de Inferencia:** ~50ms por predicción.
- **Memoria Requerida:** ~200MB.

### Pipeline de Procesamiento
1.  **Video Input:** Validación del formato de video (`.mp4`, `.avi`, `.mov`).
2.  **Extracción con MediaPipe:** Se extraen 33 landmarks de pose.
3.  **Feature Engineering:** Se generan 109 características (geométricas y temporales).
4.  **Normalización:** Se aplica `StandardScaler` a las características.
5.  **Reducción de Dimensionalidad:** Se utiliza PCA para reducir a 19 componentes, reteniendo el 95.1% de la varianza.
6.  **Inferencia del Modelo:** El MLP clasifica la actividad.
7.  **Output:** Se genera un JSON con el resultado y el nivel de confianza.

---

## 🌐 Arquitectura de la API REST

### Endpoints Principales

#### `POST /predict/video`
Clasifica un video completo.

- **Input:**
  - `video_file`: `multipart/form-data`
  - `options`: `{"frame_skip": 5, "confidence_threshold": 0.7}`
- **Respuesta de ejemplo:**
  ```json
  {
    "prediction": {
      "activity": "caminar_hacia",
      "confidence": 0.87,
      "frame_predictions": ["caminar_hacia", "caminar_hacia", "girar"],
      "temporal_consistency": 0.92
    },
    "metadata": {
      "processing_time_ms": 2340,
      "frames_processed": 120,
      "landmarks_detected_avg": 32.8
    }
  }
  ```

#### `POST /predict/frame`
Clasifica un único frame (imagen).

- **Input:**
  - `image_file`: `multipart/form-data`
- **Respuesta de ejemplo:**
  ```json
  {
    "prediction": {
      "activity": "sentarse",
      "confidence": 0.94,
      "all_probabilities": {
        "caminar_hacia": 0.02,
        "caminar_regreso": 0.01,
        "girar": 0.03,
        "ponerse_pie": 0.00,
        "sentarse": 0.94
      }
    },
    "metadata": {
      "processing_time_ms": 78,
      "landmarks_detected": 33
    }
  }
  ```

#### `POST /predict/stream`
Permite el procesamiento en tiempo real a través de WebSocket.

- **Conexión:** `ws://localhost:8000/predict/stream`
- **Input:** Frames codificados en Base64.
- **Tasa de Respuesta:** ~20 FPS.
- **Latencia:** ~50ms.

#### `GET /health` y `GET /model/info`
Endpoints para monitoreo y verificación del estado del servicio.

- **Respuesta de `/health`:**
  ```json
  {
    "status": "healthy",
    "model_loaded": true,
    "response_time_ms": 5
  }
  ```
- **Respuesta de `/model/info`:**
  ```json
  {
    "model_version": "v1.0_corrected",
    "accuracy": 0.85,
    "features": 19,
    "classes": 5,
    "trained_date": "2025-11-02"
  }
  ```

---

## 🐳 Containerización con Docker

### Dockerfile
```dockerfile
# Dockerfile para Sistema de Clasificación de Actividades
FROM python:3.9-slim

# Metadata
LABEL maintainer="tomas@proyecto-ia.com"
LABEL version="1.0"
LABEL description="API REST para clasificación de actividades"

# Dependencias del sistema operativo
RUN apt-get update && apt-get install -y \
    libgl1-mesa-glx \
    libglib2.0-0 \
    ffmpeg \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Crear un usuario no-root para seguridad
RUN useradd --create-home --shell /bin/bash app_user
WORKDIR /app

# Instalar dependencias de Python
COPY requirements.txt .
RUN pip install --no-cache-dir --upgrade pip
RUN pip install --no-cache-dir -r requirements.txt

# Copiar archivos de la aplicación
COPY models/ ./models/
COPY transformers/ ./transformers/
COPY src/ ./src/
COPY api/ ./api/

# Asignar permisos al usuario
RUN chown -R app_user:app_user /app
USER app_user

# Variables de entorno
ENV PYTHONPATH=/app
ENV MODEL_PATH=/app/models/model_clean_final.pkl
ENV SCALER_PATH=/app/transformers/scaler.pkl
ENV ENCODER_PATH=/app/transformers/encoder.pkl

# Health check para el contenedor
HEALTHCHECK --interval=30s --timeout=10s --retries=3 \
    CMD curl -f http://localhost:8000/health || exit 1

EXPOSE 8000

# Comando de inicio
CMD ["uvicorn", "api.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

### `requirements.txt`
```txt
# Core API
fastapi==0.104.1
uvicorn[standard]==0.24.0
pydantic==2.5.0

# ML & Computer Vision
mediapipe==0.10.7
scikit-learn==1.3.2
numpy==1.24.3
opencv-python-headless==4.8.1.78
joblib==1.3.2

# Utilities
python-multipart==0.0.6
aiofiles==23.2.1
pillow==10.0.1

# Monitoring
prometheus-client==0.19.0
structlog==23.2.0
```

### `docker-compose.yml`
```yaml
version: '3.8'

services:
  activity-api:
    build: .
    image: video-activity-classifier:v1.0
    container_name: activity-api
    ports:
      - "8000:8000"
    volumes:
      - ./logs:/app/logs
      - ./uploads:/app/uploads
    environment:
      - ENVIRONMENT=production
      - LOG_LEVEL=info
      - MAX_UPLOAD_SIZE=50MB
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
    restart: unless-stopped
    networks:
      - activity-network

  nginx:
    image: nginx:alpine
    container_name: activity-nginx
    ports:
      - "80:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf
    depends_on:
      - activity-api
    restart: unless-stopped
    networks:
      - activity-network

  prometheus:
    image: prom/prometheus
    container_name: activity-prometheus
    ports:
      - "9090:9090"
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml
    networks:
      - activity-network

networks:
  activity-network:
    driver: bridge
```

---

## 📊 Monitoreo y Logging

### Métricas Críticas
- **Performance:**
  - `response_time_ms`: < 200ms
  - `throughput`: Peticiones por minuto.
  - `cpu_usage`: < 70%
  - `memory_usage`: < 500MB
- **Calidad del Modelo:**
  - `prediction_confidence_avg`: > 0.80
  - `mediapipe_detection_rate`: > 90%
  - `error_rate`: < 5%
  - `model_accuracy_drift`: Monitoreo contra el 85% de base.
- **Negocio:**
  - `daily_requests`: Conteo total de peticiones.
  - `activity_distribution`: Balance de las clases predichas.
  - `uptime`: > 99.5%

### Alertas Configuradas
- **CRÍTICAS:**
  - `accuracy_drop` < 80% (posible data drift).
  - `error_rate` > 10% (investigar causa raíz).
  - `response_time` > 500ms (optimizar o escalar recursos).
- **ADVERTENCIAS:**
  - `confidence_low` < 0.75 (promedio).
  - `disk_space` > 80% de uso.

### Estructura de Logs (JSON)
```json
{
  "timestamp": "2025-11-02T17:05:00Z",
  "request_id": "req_abc123",
  "endpoint": "/predict/video",
  "processing_pipeline": {
    "mediapipe_ms": 45,
    "feature_engineering_ms": 12,
    "model_inference_ms": 23,
    "total_ms": 80
  },
  "input": {
    "type": "video_file",
    "size_mb": 2.3
  },
  "prediction": {
    "activity": "caminar_hacia",
    "confidence": 0.87
  },
  "status": "success"
}
```

---

## 🚀 Comandos de Despliegue

### Build y Ejecución
```bash
# 1. Construir la imagen de Docker
docker build -t video-activity-classifier:v1.0 .

# 2. Desplegar la orquestación completa
docker-compose up -d

# 3. Verificar el estado del servicio
curl http://localhost:80/health

# 4. Enviar una predicción de prueba
curl -X POST "http://localhost:80/predict/frame" \
  -H "Content-Type: multipart/form-data" \
  -F "image_file=@test_frame.jpg"

# 5. Monitorear los logs en tiempo real
docker logs -f activity-api

# 6. Escalar el servicio si es necesario
docker-compose up -d --scale activity-api=3
```

### Configuración de Nginx (`nginx.conf`)
```nginx
events {
    worker_connections 1024;
}

http {
    # Define el grupo de servidores de la API
    upstream activity_api {
        server activity-api:8000;
    }
    
    server {
        listen 80;
        client_max_body_size 50M; # Límite para subida de videos
        
        location / {
            proxy_pass http://activity_api;
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_connect_timeout 60s;
            proxy_read_timeout 60s;
        }
    }
}
```

---

## 🎯 Análisis de Impactos

### Impacto Técnico
- **Positivo:** Sistema automatizado, escalable y con capacidad de procesamiento en tiempo real.
- **Riesgo:** Alta dependencia en la librería MediaPipe y una precisión del 85% que puede no ser suficiente para todos los casos de uso.
- **Mitigación:** Implementar fallbacks, monitorear la precisión continuamente y planificar reentrenamientos periódicos.

### Impacto Social
- **Aplicaciones:** Potencial en fisioterapia, análisis deportivo, medicina y fitness.
- **Beneficios:** Permite un análisis objetivo del movimiento humano y puede asistir en procesos de rehabilitación.
- **Consideraciones:** Es crucial gestionar la privacidad de los datos biométricos de los usuarios.

### Impacto Ético
- **Privacidad:** Los videos deben ser procesados localmente o en memoria sin almacenamiento persistente para proteger al usuario.
- **Sesgo:** El modelo fue entrenado con un dataset limitado, lo que podría generar sesgos demográficos o de otro tipo.
- **Transparencia:** El modelo es interpretable (hasta cierto punto) y la API reporta la confianza de cada predicción.

### Plan de Mantenimiento
- **Reentrenamiento:** Programado cada 6 meses con nuevos datos para evitar el *model drift*.
- **Monitoreo de Drift:** Alertas automáticas para detectar degradación en el rendimiento del modelo.
- **Actualizaciones:** Mantener actualizadas las dependencias críticas (MediaPipe, FastAPI) para corregir bugs y vulnerabilidades.
- **Backups:** Realizar copias de seguridad de modelos, configuraciones y datos críticos.

---

## ✅ Checklist de Despliegue

### Pre-Despliegue
- [x] Modelo validado sin data leakage.
- [x] Imagen de Docker construida y probada localmente.
- [x] Endpoints de la API funcionando correctamente.
- [x] Health checks y logs estructurados implementados.

### Despliegue
- [ ] Contenedores desplegados con `docker-compose`.
- [ ] Proxy inverso (Nginx) configurado y funcionando.
- [ ] Monitoreo con Prometheus activo.
- [ ] Certificados SSL/HTTPS configurados para producción.
- [ ] Backups automáticos habilitados.

### Post-Despliegue
- [ ] Pruebas de carga (load testing) realizadas.
- [ ] Sistema de alertas verificado.
- [ ] Dashboard de monitoreo disponible para el equipo.
- [ ] Documentación de la API actualizada.
- [ ] Plan de rollback preparado y validado.

---

> **Preparado por:** Tomas  
> **Fecha:** 02 de Noviembre, 2025  
> **Status:** ✅ Plan de Despliegue Completo.
```